In [ ]:
import pandas as pd
import numpy as np

In [ ]:
def extract(x: pd.DataFrame) -> str:
    result = str()
    first = np.array([])
    second = np.array([])
    third = np.array(x[2])
    first = np.array(x[0], dtype = int)/10000
    second = np.array(x[1], dtype = int)/10000
    for j in range(0, third.size):
        result += f'{first[j]:.4f}\t{second[j]:.4f}\t{third[j]}\n'
    return result

In [ ]:
columns = ['index','Trial','Event Type','Code','Time','TTime','Uncertainty','Duration','Uncertainty','ReqTime','ReqDur','Stim Type','Pair Index']
for sub_index in range(1,24):
    for round_index in ['4fMRI_r1', '_r24fMRI']:
        filename = ''
        if(round_index == '4fMRI_r1'):
            filename = ''
        if(round_index == '_r24fMRI'):
            filename = '-round2'
        with open('./ER'+f'{sub_index:03}'+'-ResCDT_fMRI'+filename+'.log','r') as df:
                log = df.read()
                logs = log.split("ER"+f'{sub_index:03}'+";")[1:]
                data = pd.DataFrame([], columns=columns)
                for i in range(0, len(logs)):
                    if(logs[i].startswith('Type;Code;')):
                        break
                    if(i == len(logs) -1):
                        line = (logs[-1].split('Event Type'))[0].split(';')
                    else:
                        line = logs[i].split(';')
                    while(len(line) != len(columns[1:])):
                        line.append('')
                    print(line)
                    temp = [i]
                    temp = temp + line
                    print(temp)
                    data = data.append(pd.Series(temp, index=columns),ignore_index=True)
                    
        rate1, rate2, rate3, rate4 = ([] for i in range(4))
        cong1, cong2, cong3, cong4 =([] for i in range(4))
        primePic = []
        targetPic = []
        ratePic = []
        r = ''
        if(round_index == '_r24fMRI'):
            r = 'r2'
        if(round_index == '4fMRI_r1'):
            r = 'r1'

        accTime = []
        startIndex = 0
        startTime = 0
        firstPulse = False

        for i in range(data.shape[0]):
            if(firstPulse == False and data['Event Type'][i] != 'Pulse'):
                accTime.append(0)
            elif(firstPulse == False and data['Event Type'][i] == 'Pulse'):
                firstPulse = True
                startIndex = i
                startTime = data['Time'][i]
                accTime.append(0)
            else:
                accTime.append((int(data['Time'][i])-int(startTime)))

        data['Accumulated Time'] = accTime

        for i in range(data.shape[0]):
            if(data['Code'][i] == 'primeStim'):
                primePic.append([int(data['Accumulated Time'][i]), int(data['ReqDur'][i]),1])
        primePic = pd.DataFrame(primePic)
        result = extract(primePic)
        try:
            with open("./timefile/ER"+f'{sub_index:03}'+"_primePic_"+r+".txt", 'x') as f:
                f.write(result)
        except:
            print('file exist')

        for i in range(data.shape[0]):
            if(data['Code'][i] == 'target'):
                targetPic.append([int(data['Accumulated Time'][i]), int(data['ReqDur'][i]),1])
        targetPic = pd.DataFrame(targetPic)
        result = extract(targetPic)
        try:
            with open("./timefile/ER"+f'{sub_index:03}'+"_targetPic_"+r+".txt", 'x') as f:
                f.write(result)
        except:
            print('file exist')

        for i in range(data.shape[0]):
            if(data['Code'][i] == 'ratingStim_trial'):
                ratePic.append([int(data['Accumulated Time'][i]), int(data['ReqDur'][i]),1])
        ratePic = pd.DataFrame(ratePic)
        result = extract(ratePic)
        try:
            with open("./timefile/ER"+f'{sub_index:03}'+"_ratePic_"+r+".txt", 'x') as f:
                f.write(result)
        except:
            print('file exist')

        cong = []
        rate = []
        startTime = 0
        etype = 0
        for i in range(startIndex, data.shape[0]):
            if(data['Code'][i] == 'target'):
                etype = 1
                startTime = int(data['Accumulated Time'][i])

            if(data['Code'][i] == 'ratingStim_trial'):
                etype = 2
                startTime = int(data['Accumulated Time'][i])

            if(data['Event Type'][i] == 'Response' and etype == 1):
                cong.append([int(data['Accumulated Time'][i]), (int(data['Accumulated Time'][i]) - startTime),1, data['Code'][i]])
                etype = 0
            if(data['Event Type'][i] == 'Response' and etype == 2):
                rate.append([int(data['Accumulated Time'][i]), (int(data['Accumulated Time'][i]) - startTime),1, data['Code'][i]])
                etype = 0

        cong = pd.DataFrame(cong)
        rate = pd.DataFrame(rate)
        logfile = pd.read_csv("./ER"+f'{sub_index:03}'+"_resCDT_formal"+round_index+".txt")
        cong[3] = pd.Series((logfile[logfile['Targetnum'] != 2].reset_index())['congruType'])
        rate[3] = pd.Series((logfile[logfile['Targetnum'] != 2].reset_index())['EmoType'])

        rate2 = rate[rate[3] == 2]
        rate3 = rate[rate[3] == 3]
        rate4 = rate[rate[3] == 4]
        cong2 = cong[cong[3] == 2]
        cong3 = cong[cong[3] == 3]
        cong4 = cong[cong[3] == 4]

        result = extract((cong[cong[3] == 2])[[0,1,2]])
        try:
            with open("./timefile/ER"+f'{sub_index:03}'+"_cong2_"+r+".txt", 'x') as f:
                f.write(result)
        except:
            print('file exist')

        result = extract((cong[cong[3] == 3])[[0,1,2]])
        try:
            with open("./timefile/ER"+f'{sub_index:03}'+"_cong3_"+r+".txt", 'x') as f:
                f.write(result)
        except:
            print('file exist')

        result = extract((cong[cong[3] == 4])[[0,1,2]])
        try:
            with open("./timefile/ER"+f'{sub_index:03}'+"_cong4_"+r+".txt", 'x') as f:
                f.write(result)
        except:
            print('file exist')

        result = extract((rate[rate[3] == 2])[[0,1,2]])
        try:
            with open("./timefile/ER"+f'{sub_index:03}'+"_rate2_"+r+".txt", 'x') as f:
                f.write(result)
        except:
            print('file exist')

        result = extract((rate[rate[3] == 3])[[0,1,2]])
        try:
            with open("./timefile/ER"+f'{sub_index:03}'+"_rate3_"+r+".txt", 'x') as f:
                f.write(result)
        except:
            print('file exist')

        result = extract((rate[rate[3] == 4])[[0,1,2]])
        try:
            with open("./timefile/ER"+f'{sub_index:03}'+"_rate4_"+r+".txt", 'x') as f:
                f.write(result)
        except:
            print('file exist')